In [4]:
# =========================================
# VWAPクロス × 残差フィルター（BTC基準）バックテスト
# グリッドサーチ版：
#   COEFF_WINDOW / VWAP_RESID_WINDOW / RESID_K / VWAP_SHORT / VWAP_MEDIUM を探索
# 上位10パターンの最終リターンと設定をコンソール出力
# =========================================

# 0) Libraries
import pandas as pd
import numpy as np
from itertools import product

# 1) Strategy / BT Parameters  ----------------------------------------------
# データ
BTC_csv = 'Market_Bybit_BTCUSDT_5min_20250101-20250730'
ALT_csv = 'Market_Bybit_SOLUSDT_5min_20250101-20250730'
time_col = "timestamp"

# 固定パラメータ（※グリッド対象外）
USE_MAD_SCALE = False        # True: MAD×1.4826 / False: 標準偏差
RESID_PERSIST = 1            # 連続本数（1=一発、2=2本連続）
USE_DIRECTION_FILTER = True  # 残差符号とVWAPレジーム方向の一致要求

# BT
INITIAL_CAPITAL = 100.0
TAKER_FEE = 0.0004           # 片道手数料
NOTIONAL_SIZE = 500.0        # 名目USD
MIN_QTY = 0.0                # 取引所の最小数量（無ければ0）

# ── グリッド探索レンジ（必要に応じて調整） ─────────────────────────
COEFF_WINDOW_grid       = [1, 3, 5, 7, 10, 15]      # 残差回帰・スケール窓
VWAP_RESID_WINDOW_grid  = [1, 2, 3, 4, 5]           # 残差用VWAP窓
RESID_K_grid            = [0.5, 1.0, 1.5, 2.0]      # |z| > K
VWAP_SHORT_grid         = [5, 7, 10, 12]            # 例：短期VWAP候補
VWAP_MEDIUM_grid        = [12, 14, 16, 20, 24]      # 例：中期VWAP候補（SHORTより大きいものだけ採用）

# 2) IO & Prep  ------------------------------------------------------------
def load_market(csv, rename_prefix=None):
    df = pd.read_csv(f'../01.data/{csv}.csv')
    df[time_col] = pd.to_datetime(df[time_col])
    df = df.sort_values(time_col)
    if rename_prefix:
        rename = {c: f"{c}_{rename_prefix}" for c in ["O","H","L","C","V"] if c in df.columns}
        df = df.rename(columns=rename)
    return df

df_btc = load_market(BTC_csv)                     # timestamp, O/H/L/C/V
df_altOHLCV = load_market(ALT_csv, "ALT")         # timestamp, O_ALT/H_ALT/L_ALT/C_ALT/V_ALT

# 3) 残差（ALT vs BTC）を「VWAP_resid のリターン」で算出  ------------------
def rolling_vwap_from_ohlcv(o, h, l, c, v, window):
    w = int(window)
    typical = (o + h + l + c) / 4.0
    tpv = typical * v
    num = tpv.rolling(w, min_periods=w).sum()
    den = v.rolling(w, min_periods=w).sum()
    return num / den

def compute_residuals_via_vwap(
    df_alt: pd.DataFrame,
    df_btc: pd.DataFrame,
    vwap_resid_window: int,
    coeff_window: int,
    use_mad_scale: bool
) -> pd.DataFrame:
    # ALT/BTC: 残差用 rolling VWAP
    vwap_alt = rolling_vwap_from_ohlcv(
        df_alt["O_ALT"], df_alt["H_ALT"], df_alt["L_ALT"], df_alt["C_ALT"], df_alt["V_ALT"],
        vwap_resid_window
    )
    vwap_btc = rolling_vwap_from_ohlcv(
        df_btc["O"], df_btc["H"], df_btc["L"], df_btc["C"], df_btc["V"],
        vwap_resid_window
    )

    x = df_alt[[time_col]].copy()
    x["VWAP_resid_ALT"] = vwap_alt.values
    x = x.merge(
        df_btc[[time_col]].assign(VWAP_resid_BTC=vwap_btc.values),
        on=time_col, how="inner"
    )

    # 対数リターン（VWAP_residベース）
    x["ret_ALT"] = np.log(x["VWAP_resid_ALT"]).diff()
    x["ret_BTC"] = np.log(x["VWAP_resid_BTC"]).diff()
    x = x.dropna(subset=["ret_ALT", "ret_BTC"]).reset_index(drop=True)

    # ローリングOLS：beta = Cov/Var, alpha = mean_ALT - beta*mean_BTC
    W = int(coeff_window)
    mean_ALT = x["ret_ALT"].rolling(W, min_periods=W).mean()
    mean_BTC = x["ret_BTC"].rolling(W, min_periods=W).mean()
    var_BTC  = x["ret_BTC"].rolling(W, min_periods=W).var(ddof=0)
    cov_AB   = x["ret_ALT"].rolling(W, min_periods=W).cov(x["ret_BTC"])

    with np.errstate(divide='ignore', invalid='ignore'):
        x["beta_BTC"] = cov_AB / var_BTC
    x["alpha"]    = mean_ALT - x["beta_BTC"] * mean_BTC

    # 予想 & 残差
    x["ret_ALT_pred"] = x["alpha"] + x["beta_BTC"] * x["ret_BTC"]
    x["resid"] = x["ret_ALT"] - x["ret_ALT_pred"]

    # 標準化（Z）
    if use_mad_scale:
        def _mad(s):
            m = s.median()
            return 1.4826 * (np.abs(s - m)).median()
        scale = x["resid"].rolling(W, min_periods=W).apply(_mad, raw=False)
    else:
        scale = x["resid"].rolling(W, min_periods=W).std()

    x["resid_z"] = x["resid"] / scale
    return x[[time_col, "resid_z"]]

# 4) ALT側でVWAPクロス（従来どおり） & 残差フィルター合成  -----------------
def add_vwap_and_signal(
    df_alt: pd.DataFrame,
    resid_df: pd.DataFrame,
    resid_k: float,
    resid_persist: int,
    use_direction_filter: bool,
    vwap_short: int,
    vwap_medium: int
) -> pd.DataFrame:
    d = df_alt.merge(resid_df[[time_col, "resid_z"]], on=time_col, how="left").copy()

    # エントリー用 VWAP（短/中）
    vs, vm = int(vwap_short), int(vwap_medium)
    d["Typical_price"] = (d["O_ALT"] + d["H_ALT"] + d["L_ALT"] + d["C_ALT"]) / 4
    d["TPxV"] = d["Typical_price"] * d["V_ALT"]

    roll_tpv_s = d["TPxV"].rolling(vs,  min_periods=1).sum()
    roll_v_s   = d["V_ALT"].rolling(vs,  min_periods=1).sum()
    roll_tpv_m = d["TPxV"].rolling(vm, min_periods=1).sum()
    roll_v_m   = d["V_ALT"].rolling(vm, min_periods=1).sum()

    d["VWAP_short"]  = roll_tpv_s / roll_v_s
    d["VWAP_medium"] = roll_tpv_m / roll_v_m

    # クロス Regime（1: long bias / 0: short bias）
    d["Regeme_vwap"] = (d["VWAP_short"] > d["VWAP_medium"]).astype(int)
    reg_prev = d["Regeme_vwap"].shift(1)
    reg_changed = (d["Regeme_vwap"] != reg_prev).fillna(False)

    # 点火バー（クロス瞬間）
    d["Trade_ignittion_vwap"] = np.where(reg_changed, d["Regeme_vwap"], np.nan)

    # 残差フィルター
    gate_abs = d["resid_z"].abs() > float(resid_k)
    gate_persist = gate_abs.rolling(int(resid_persist), min_periods=int(resid_persist)).sum() == int(resid_persist)

    if use_direction_filter:
        gate_dir = ((d["resid_z"] > 0) & (d["Regeme_vwap"] == 1)) | \
                   ((d["resid_z"] < 0) & (d["Regeme_vwap"] == 0))
        gate = gate_persist & gate_dir
    else:
        gate = gate_persist

    # 最終Trade（発火バー ∧ フィルター通過）
    d["Trade"] = np.where(d["Trade_ignittion_vwap"].notna() & gate, d["Regeme_vwap"], np.nan)

    # 参考：フィルター無し版（比較用）
    d["Trade_vwap_only"] = np.where(d["Trade_ignittion_vwap"].notna(), d["Regeme_vwap"], np.nan)
    return d

# 5) バックテスト  ---------------------------------------------------------
def run_backtest(
    df_in: pd.DataFrame,
    initial_capital: float = INITIAL_CAPITAL,
    fee: float = TAKER_FEE,
    notional_size: float = NOTIONAL_SIZE,
    min_qty: float = MIN_QTY
) -> pd.DataFrame:
    """名目サイズ固定・線形PnL・片道手数料・ドテン対応"""
    d = df_in.rename(columns={"C_ALT":"C","V_ALT":"V"}).copy()
    d = d[[time_col, "C", "V", "Trade"]].copy().sort_values(time_col).reset_index(drop=True)

    n = len(d)
    side = np.zeros(n, dtype=int)
    pos_qty = np.zeros(n)
    entry_price = np.full(n, np.nan)
    trade_qty = np.zeros(n)
    trade_notional = np.zeros(n)
    fee_paid = np.zeros(n)
    fee_cum = np.zeros(n)
    realized_pnl = np.zeros(n)
    realized_pnl_cum = np.zeros(n)
    unrealized_pnl = np.zeros(n)
    equity = np.zeros(n)

    cur_qty = 0.0
    cur_entry = np.nan
    cur_fee_cum = 0.0
    cur_realized = 0.0

    for i in range(n):
        price = float(d.at[i, "C"])
        sig = d.at[i, "Trade"]  # 1/0/NaN

        # 取引判定
        do_trade = False
        desired_side = 0
        if pd.notna(sig):
            desired_side = 1 if sig == 1 else -1
            if cur_qty == 0.0:
                do_trade = True
            else:
                cur_side = 1 if cur_qty > 0 else -1
                if desired_side == -cur_side:
                    do_trade = True

        # 約定
        if do_trade:
            # 既存クローズ
            if cur_qty != 0.0:
                close_qty = -cur_qty
                pnl_close = (price - cur_entry) * cur_qty
                cur_realized += pnl_close
                close_notional = abs(close_qty) * price
                fee_close = fee * close_notional
                cur_fee_cum += fee_close

                trade_qty[i] += close_qty
                trade_notional[i] += close_notional
                fee_paid[i] += fee_close
                realized_pnl[i] += pnl_close

                cur_qty = 0.0
                cur_entry = np.nan

            # 新規
            target_qty = notional_size / price
            if min_qty and min_qty > 0:
                target_qty = np.floor(target_qty / min_qty) * min_qty
            if target_qty > 0:
                open_qty = desired_side * target_qty
                open_notional = abs(open_qty) * price
                fee_open = fee * open_notional
                cur_fee_cum += fee_open

                trade_qty[i] += open_qty
                trade_notional[i] += open_notional
                fee_paid[i] += fee_open

                cur_qty = open_qty
                cur_entry = price

        # 評価
        u_pnl = (price - cur_entry) * cur_qty if cur_qty != 0 else 0.0
        unrealized_pnl[i] = u_pnl

        if i > 0:
            realized_pnl_cum[i] = realized_pnl_cum[i-1] + realized_pnl[i]
            fee_cum[i] = fee_cum[i-1] + fee_paid[i]
        else:
            realized_pnl_cum[i] = realized_pnl[i]
            fee_cum[i] = fee_paid[i]

        equity[i] = initial_capital + realized_pnl_cum[i] + unrealized_pnl[i] - cur_fee_cum

        side[i] = 0 if cur_qty == 0 else (1 if cur_qty > 0 else -1)
        pos_qty[i] = cur_qty
        entry_price[i] = cur_entry

    out = d.copy()
    out["side"] = side
    out["position_qty"] = pos_qty
    out["entry_price"] = entry_price
    out["trade_qty"] = trade_qty
    out["trade_notional"] = trade_notional
    out["fee_paid"] = fee_paid
    out["fee_paid_cum"] = fee_cum
    out["realized_pnl"] = realized_pnl
    out["realized_pnl_cum"] = realized_pnl_cum
    out["unrealized_pnl"] = unrealized_pnl
    out["equity"] = equity
    return out

# 6) 評価補助 --------------------------------------------------------------
def max_dd_while_in_position(bt: pd.DataFrame, time_col: str) -> dict:
    in_pos = bt['position_qty'].ne(0)
    peak = None
    run_start_ts = None
    max_dd = 0.0
    worst_start_ts = None
    worst_trough_ts = None

    for i, row in bt.iterrows():
        if in_pos.iloc[i]:
            eq = float(row['equity'])
            ts = row[time_col]
            if peak is None:
                peak = eq
                run_start_ts = ts
            if eq > peak:
                peak = eq
            dd = (eq / peak) - 1.0
            if dd < max_dd:
                max_dd = dd
                worst_start_ts = run_start_ts
                worst_trough_ts = ts
        else:
            peak = None
            run_start_ts = None

    return {
        'max_dd_frac': max_dd,
        'max_dd_pct' : max_dd * 100.0,
        'start_ts'   : worst_start_ts,
        'trough_ts'  : worst_trough_ts,
    }

# 7) グリッドサーチ本体（残差の再計算はキャッシュ） ------------------------
results = []
resid_cache = {}  # key = (coeff_w, vwap_resid_w) -> resid_df

for coeff_w, vwap_resid_w in product(COEFF_WINDOW_grid, VWAP_RESID_WINDOW_grid):
    key = (int(coeff_w), int(vwap_resid_w))
    # 残差をキャッシュ生成
    if key not in resid_cache:
        resid_df = compute_residuals_via_vwap(
            df_altOHLCV[[time_col, "O_ALT", "H_ALT", "L_ALT", "C_ALT", "V_ALT"]],
            df_btc[[time_col, "O", "H", "L", "C", "V"]],
            vwap_resid_window=vwap_resid_w,
            coeff_window=coeff_w,
            use_mad_scale=USE_MAD_SCALE
        )
        # 全NaN（Wや窓の関係）ならスキップ用に記録
        if resid_df["resid_z"].notna().sum() == 0:
            resid_cache[key] = None
        else:
            resid_cache[key] = resid_df

    cached_resid = resid_cache[key]
    if cached_resid is None:
        continue  # 有効な残差が作れなかった組は飛ばす

    # 短期・中期VWAPの組をループ（SHORT < MEDIUM のみ）
    for vwap_s in VWAP_SHORT_grid:
        for vwap_m in VWAP_MEDIUM_grid:
            if int(vwap_s) >= int(vwap_m):
                continue

            # 残差閾値もループ
            for resid_k in RESID_K_grid:
                df_all = add_vwap_and_signal(
                    df_altOHLCV, cached_resid,
                    resid_k=resid_k,
                    resid_persist=RESID_PERSIST,
                    use_direction_filter=USE_DIRECTION_FILTER,
                    vwap_short=vwap_s,
                    vwap_medium=vwap_m
                )

                df_bt = df_all[[time_col, "C_ALT", "V_ALT", "Trade"]]
                bt = run_backtest(df_bt, INITIAL_CAPITAL, TAKER_FEE, NOTIONAL_SIZE, MIN_QTY)

                final_eq = float(bt['equity'].iloc[-1])
                total_return = (final_eq / INITIAL_CAPITAL) - 1.0
                total_fee = float(bt['fee_paid'].sum())
                trades = int((bt['trade_qty'] != 0).sum())
                mdd = max_dd_while_in_position(bt, time_col)['max_dd_pct']

                results.append({
                    'COEFF_WINDOW': coeff_w,
                    'VWAP_RESID_WINDOW': vwap_resid_w,
                    'RESID_K': resid_k,
                    'VWAP_SHORT': vwap_s,
                    'VWAP_MEDIUM': vwap_m,
                    'final_equity': final_eq,
                    'total_return_pct': total_return * 100.0,
                    'total_fee_usd': total_fee,
                    'trade_bars': trades,
                    'max_dd_inpos_pct': mdd
                })

# 8) 上位10件を表示 --------------------------------------------------------
res_df = pd.DataFrame(results)
if len(res_df) == 0:
    print("有効な結果が得られませんでした。窓長（特にCOEFF_WINDOWやVWAP_RESID_WINDOW）を見直してください。")
else:
    res_df = res_df.sort_values(['final_equity','total_return_pct'], ascending=False).reset_index(drop=True)

    topN = min(10, len(res_df))
    print("\n========== Grid Search Top Results (by Final Equity) ==========")
    for i in range(topN):
        r = res_df.iloc[i]
        print(
            f"[{i+1:02d}] "
            f"Ret={r['total_return_pct']:+7.2f}% | Eq={r['final_equity']:8.2f} | "
            f"coefW={int(r['COEFF_WINDOW'])}, vwapResidW={int(r['VWAP_RESID_WINDOW'])}, "
            f"K={r['RESID_K']:.2f} | VWAP(S/M)={int(r['VWAP_SHORT'])}/{int(r['VWAP_MEDIUM'])} | "
            f"trades={int(r['trade_bars'])}, fee={r['total_fee_usd']:.2f} USD, "
            f"MDD_inpos={r['max_dd_inpos_pct']:.2f}%"
        )

    # 参考：ベスト設定（必要なら変数として保持）
    best = res_df.iloc[0]
    BEST_COEFF_WINDOW = int(best['COEFF_WINDOW'])
    BEST_VWAP_RESID_WINDOW = int(best['VWAP_RESID_WINDOW'])
    BEST_RESID_K = float(best['RESID_K'])
    BEST_VWAP_SHORT = int(best['VWAP_SHORT'])
    BEST_VWAP_MEDIUM = int(best['VWAP_MEDIUM'])
    # print("\nBest Params:", BEST_COEFF_WINDOW, BEST_VWAP_RESID_WINDOW, BEST_RESID_K, BEST_VWAP_SHORT, BEST_VWAP_MEDIUM)



========== Grid Search Top Results (by Final Equity) ==========
[01] Ret=+987.35% | Eq= 1087.35 | coefW=3, vwapResidW=5, K=2.00 | VWAP(S/M)=5/20 | trades=64, fee=25.46 USD, MDD_inpos=-103.08%
[02] Ret=+940.32% | Eq= 1040.32 | coefW=15, vwapResidW=5, K=2.00 | VWAP(S/M)=10/12 | trades=198, fee=79.06 USD, MDD_inpos=-46.13%
[03] Ret=+936.26% | Eq= 1036.26 | coefW=5, vwapResidW=1, K=1.50 | VWAP(S/M)=12/16 | trades=191, fee=76.23 USD, MDD_inpos=-102.02%
[04] Ret=+925.38% | Eq= 1025.38 | coefW=7, vwapResidW=2, K=2.00 | VWAP(S/M)=5/24 | trades=100, fee=39.81 USD, MDD_inpos=-27.84%
[05] Ret=+831.05% | Eq=  931.05 | coefW=3, vwapResidW=1, K=1.50 | VWAP(S/M)=7/20 | trades=180, fee=71.85 USD, MDD_inpos=-75.87%
[06] Ret=+785.74% | Eq=  885.74 | coefW=7, vwapResidW=1, K=1.00 | VWAP(S/M)=7/14 | trades=522, fee=208.64 USD, MDD_inpos=-59.63%
[07] Ret=+753.34% | Eq=  853.34 | coefW=7, vwapResidW=2, K=1.50 | VWAP(S/M)=12/16 | trades=204, fee=81.44 USD, MDD_inpos=-66.61%
[08] Ret=+747.85% | Eq=  847.85 |